In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

from sklearn.metrics import precision_score, recall_score, roc_curve
from sklearn.linear_model import LogisticRegression

In [147]:

from my_modul import *

In [50]:
mydata = pd.read_csv('LAQN.csv')

In [51]:
mydata.head()


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location
0,CD1,Camden - Swiss Cottage,Kerbside,09/07/2016 09:00:00 AM,Day 191 09:00,Low,1,No,No,E05000133,Frognal and Fitzjohns,526633,184391,-0.175284,51.544219,Unknown,09/07/2016 11:05:05 AM,"(51.544219, -0.175284)"
1,BL0,Camden - Bloomsbury,Urban Background,21/09/2017 01:00:00 AM,Day 264 01:00,No data,0,No,No,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,Unknown,21/09/2017 03:05:06 AM,"(51.522287, -0.125847)"
2,CD1,Camden - Swiss Cottage,Kerbside,31/05/2017 12:00:00 PM,Day 151 12:00,Low,1,No,No,E05000133,Frognal and Fitzjohns,526633,184391,-0.175284,51.544219,Unknown,31/05/2017 02:05:08 PM,"(51.544219, -0.175284)"
3,BL0,Camden - Bloomsbury,Urban Background,19/07/2017 07:00:00 PM,Day 200 19:00,Low,1,No,No,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,Unknown,19/07/2017 09:05:05 PM,"(51.522287, -0.125847)"
4,IM1,Camden - Holborn (inmidtown),Kerbside,06/03/2018 12:00:00 AM,Day 065 00:00,No data,0,No,No,E05000138,Holborn and Covent Garden,530527,181502,-0.120194,51.517368,Unknown,06/03/2018 01:08:18 AM,"(51.517368, -0.120194)"


In [52]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83148 entries, 0 to 83147
Data columns (total 18 columns):
Site Code                  83148 non-null object
Site Name                  83148 non-null object
Site Type                  83148 non-null object
Bulletin Date              83148 non-null object
Bulletin Date Formatted    83148 non-null object
NO2 Air Quality Band       83148 non-null object
NO2 Air Quality Index      83148 non-null int64
Latest Capture             83148 non-null object
Latest Week                83148 non-null object
Ward Code                  83132 non-null object
Ward Name                  83132 non-null object
Easting                    83148 non-null int64
Northing                   83148 non-null int64
Longitude                  83148 non-null float64
Latitude                   83148 non-null float64
Spatial Accuracy           83148 non-null object
Last Uploaded              83148 non-null object
Location                   83148 non-null object
dtypes: fl

In [53]:
mydata.describe()


,NO2 Air Quality Index,Easting,Northing,Longitude,Latitude
count,83148.000000,83148.000000,83148.000000,83148.000000,83148.000000
mean,1.119871,529291.392637,182641.790819,-0.137594,51.527896
std,0.740268,1552.864603,1086.945051,0.021990,0.010109
min,0.000000,526630.000000,181501.000000,-0.175284,51.517368
25%,1.000000,529070.500000,181904.250000,-0.140611,51.521057
50%,1.000000,530011.000000,182337.500000,-0.127310,51.524997
75%,2.000000,530224.000000,183098.000000,-0.124434,51.532036
max,6.000000,530529.000000,184392.000000,-0.120194,51.544219


In [54]:
for col in mydata.columns:
    print(col, len(mydata[col].unique()))

Site Code 4
Site Name 5
Site Type 3
Bulletin Date 20468
Bulletin Date Formatted 8745
NO2 Air Quality Band 3
NO2 Air Quality Index 7
Latest Capture 2
Latest Week 2
Ward Code 5
Ward Name 5
Easting 11
Northing 12
Longitude 5
Latitude 5
Spatial Accuracy 1
Last Uploaded 20792
Location 5


In [55]:
mydata.groupby('Latest Capture').count()


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location
Latest Capture,,,,,,,,,,,,,,,,,
No,83144,83144,83144,83144,83144,83144,83144,83144,83128,83128,83144,83144,83144,83144,83144,83144,83144
Yes,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [56]:
mydata['Latest Capture'] = mydata['Latest Capture'].replace('No',0)
mydata['Latest Capture'] = mydata['Latest Capture'].replace('Yes',1)
mydata['Latest Week'] = mydata['Latest Week'].replace('No',0)
mydata['Latest Week'] = mydata['Latest Week'].replace('Yes',1)

In [57]:
mydata.groupby('Spatial Accuracy').count()


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Last Uploaded,Location
Spatial Accuracy,,,,,,,,,,,,,,,,,
Unknown,83148,83148,83148,83148,83148,83148,83148,83148,83148,83132,83132,83148,83148,83148,83148,83148,83148


In [58]:
del mydata['Spatial Accuracy']

In [59]:

mydata.groupby('NO2 Air Quality Band').count()

,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Last Uploaded,Location
NO2 Air Quality Band,,,,,,,,,,,,,,,,
Low,66917,66917,66917,66917,66917,66917,66917,66917,66908,66908,66917,66917,66917,66917,66917,66917
Moderate,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67
No data,16164,16164,16164,16164,16164,16164,16164,16164,16157,16157,16164,16164,16164,16164,16164,16164


In [60]:
drop_na(mydata,0)

,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Capture,Latest Week,Easting,Northing,Longitude,Latitude,Last Uploaded,Location
0,CD1,Camden - Swiss Cottage,Kerbside,09/07/2016 09:00:00 AM,Day 191 09:00,Low,1,0,0,526633,184391,-0.175284,51.544219,09/07/2016 11:05:05 AM,"(51.544219, -0.175284)"
1,BL0,Camden - Bloomsbury,Urban Background,21/09/2017 01:00:00 AM,Day 264 01:00,No data,0,0,0,530123,182038,-0.125847,51.522287,21/09/2017 03:05:06 AM,"(51.522287, -0.125847)"
2,CD1,Camden - Swiss Cottage,Kerbside,31/05/2017 12:00:00 PM,Day 151 12:00,Low,1,0,0,526633,184391,-0.175284,51.544219,31/05/2017 02:05:08 PM,"(51.544219, -0.175284)"
3,BL0,Camden - Bloomsbury,Urban Background,19/07/2017 07:00:00 PM,Day 200 19:00,Low,1,0,0,530123,182038,-0.125847,51.522287,19/07/2017 09:05:05 PM,"(51.522287, -0.125847)"
4,IM1,Camden - Holborn (inmidtown),Kerbside,06/03/2018 12:00:00 AM,Day 065 00:00,No data,0,0,0,530527,181502,-0.120194,51.517368,06/03/2018 01:08:18 AM,"(51.517368, -0.120194)"
5,CD1,Camden - Swiss Cottage,Kerbside,15/07/2017 02:00:00 PM,Day 196 14:00,Low,1,0,0,526633,184391,-0.175284,51.544219,15/07/2017 04:05:07 PM,"(51.544219, -0.175284)"
6,IM1,Camden - Holborn (inmidtown),Kerbside,05/12/2017 01:00:00 AM,Day 339 01:00,No data,0,0,0,530527,181502,-0.120194,51.517368,05/12/2017 02:05:10 AM,"(51.517368, -0.120194)"
7,CD1,Camden - Swiss Cottage,Kerbside,14/07/2018 12:00:00 AM,Day 195 00:00,Low,1,0,0,526630,184392,-0.175284,51.544219,14/07/2018 02:05:11 AM,"(51.544219, -0.175284)"
8,IM1,Camden - Holborn (inmidtown),Kerbside,18/02/2017 11:00:00 AM,Day 049 11:00,Low,2,0,0,530529,181501,-0.120194,51.517368,18/02/2017 12:05:05 PM,"(51.517368, -0.120194)"
9,IM1,Camden - Holborn (Bee Midtown),Kerbside,12/09/2018 11:00:00 AM,Day 255 11:00,No data,0,0,0,530527,181502,-0.120194,51.517368,12/09/2018 01:05:14 PM,"(51.517368, -0.120194)"


In [61]:
'''only for numeric data'''
fill_na(mydata['NO2 Air Quality Index'])

,NO2 Air Quality Index
0,1
1,0
2,1
3,1
4,0
5,1
6,0
7,1
8,2
9,0


In [131]:
for i in range(100):    
    mydata['Longitude'][i*10] = np.nan
mydata.head()

C:\Users\MirolsavPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Last Uploaded,Location
0,CD1,Camden - Swiss Cottage,Kerbside,09/07/2016 09:00:00 AM,Day 191 09:00,Low,1,0,0,E05000133,Frognal and Fitzjohns,526633,184391,NaN,51.544219,09/07/2016 11:05:05 AM,"(51.544219, -0.175284)"
1,BL0,Camden - Bloomsbury,Urban Background,21/09/2017 01:00:00 AM,Day 264 01:00,No data,0,0,0,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,21/09/2017 03:05:06 AM,"(51.522287, -0.125847)"
2,CD1,Camden - Swiss Cottage,Kerbside,31/05/2017 12:00:00 PM,Day 151 12:00,Low,1,0,0,E05000133,Frognal and Fitzjohns,526633,184391,-0.175284,51.544219,31/05/2017 02:05:08 PM,"(51.544219, -0.175284)"
3,BL0,Camden - Bloomsbury,Urban Background,19/07/2017 07:00:00 PM,Day 200 19:00,Low,1,0,0,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,19/07/2017 09:05:05 PM,"(51.522287, -0.125847)"
4,IM1,Camden - Holborn (inmidtown),Kerbside,06/03/2018 12:00:00 AM,Day 065 00:00,No data,0,0,0,E05000138,Holborn and Covent Garden,530527,181502,-0.120194,51.517368,06/03/2018 01:08:18 AM,"(51.517368, -0.120194)"


In [151]:
pred1 = mydata['Latest Week'].astype('category')
df1 = mydata[['NO2 Air Quality Index','Longitude','Northing','Latitude']]
n = len(mydata['Latest Week'])

In [133]:
n = len(mydata['Latest Week'])
y = 0 
no = 0
for i in range(n):
    if mydata['Latest Week'][i] == 'Yes':
        y = y + 1 
    if mydata['Latest Week'][i] == 'No':
        no = no + 1 
print(y,n,y/n)

0 83148 0.0


In [137]:
def log_regression(data, pred):
    
    X_train, X_test, y_train, y_test = train_test_split(data, pred, test_size=0.2)

    logr = LogisticRegression()

    logr = logr.fit(X_train, y_train)

    y_pred = logr.predict(X_test)
    print(np.unique(y_pred))
    #score2 = roc_curve(y_test, y_pred)    
    #score3 = recall_score(y_test, y_pred)
    #score4 = precision_score(y_test, y_pred) 
    #,score2,score3,score4
    
    score1 = r2_score(y_test, y_pred)
    
    return score1

In [153]:
#,score2,score3,score4

#only with fill_na
df1 = fill_na(df1)
score1 = log_regression(df1,pred1)
score1
#r2 score возвращает отрицательное значение, что говорит о том что выбранные поля для предсказания не коррелируют с таргетом 



[0]


-0.008367693427115963

In [152]:

#with scaling
df1 = fill_na(df1)
df1 = scal(df1)
score1 = log_regression(df1,pred1)
score1

[0]


-0.010021257212268653

In [154]:
#with LR
#with scaling
df1 = LR(df1,pred = 'Longitude',data = 'Northing')
score1 = log_regression(df1,pred1)
score1

C:\Users\MirolsavPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  del sys.path[0]


[0]


-0.007512419726159791

In [129]:
pred = 'Longitude'
data = 'Easting'
def LR(df,pred,data):
        
    df_tmp = df
    
    df = pd.DataFrame(df)
    df = drop_na(df)
    y = np.array(df[pred])    
    X = np.array(df[data])
    
    X = np.c_[X, np.ones(X.shape[0])] 
    theta = np.linalg.lstsq(X,y)[0]
    thet_o = theta[-1]
    theta = theta[:-1]
    def listsum(numList):
        if len(numList) == 1:
            return numList[0]
        else:
            return numList[0] + listsum(numList[1:])  

    for i in range (len(df_tmp)):
        new = []
        
        if df_tmp[pred][i] != df_tmp[pred][i]:    
                              
            new.append(df_tmp[data][i])                          
                     
            res = np.multiply(np.array(new),theta)   
                         
            df_tmp[pred][i] = (res.sum + theta[-1])
            
    return df_tmp

In [130]:
LR(mydata,pred,data).head()
#посмотрев на каждое 10-ое значение можно заметить что линейная регрессия заполнила наны(правда с большой погрешностью)

C:\Users\MirolsavPC\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  del sys.path[0]


,Site Code,Site Name,Site Type,Bulletin Date,Bulletin Date Formatted,NO2 Air Quality Band,NO2 Air Quality Index,Latest Capture,Latest Week,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Last Uploaded,Location
0,CD1,Camden - Swiss Cottage,Kerbside,09/07/2016 09:00:00 AM,Day 191 09:00,Low,1,0,0,E05000133,Frognal and Fitzjohns,526633,184391,7.457635,51.544219,09/07/2016 11:05:05 AM,"(51.544219, -0.175284)"
1,BL0,Camden - Bloomsbury,Urban Background,21/09/2017 01:00:00 AM,Day 264 01:00,No data,0,0,0,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,21/09/2017 03:05:06 AM,"(51.522287, -0.125847)"
2,CD1,Camden - Swiss Cottage,Kerbside,31/05/2017 12:00:00 PM,Day 151 12:00,Low,1,0,0,E05000133,Frognal and Fitzjohns,526633,184391,-0.175284,51.544219,31/05/2017 02:05:08 PM,"(51.544219, -0.175284)"
3,BL0,Camden - Bloomsbury,Urban Background,19/07/2017 07:00:00 PM,Day 200 19:00,Low,1,0,0,E05000129,Bloomsbury,530123,182038,-0.125847,51.522287,19/07/2017 09:05:05 PM,"(51.522287, -0.125847)"
4,IM1,Camden - Holborn (inmidtown),Kerbside,06/03/2018 12:00:00 AM,Day 065 00:00,No data,0,0,0,E05000138,Holborn and Covent Garden,530527,181502,-0.120194,51.517368,06/03/2018 01:08:18 AM,"(51.517368, -0.120194)"
